## Идеи

1. Необходимо классифицировать изображения на 12 классов. Используем простую сверточную сеть с двумя полносвязными слоями. На выходе - слой размера 12 с softmax. На входе - тензор батча изобржаний
2. Изображения очень похожи друг на друга. В центре - побег растения, фон - почва (камни, замля и т.д.)
3. Изображения не стандартного формата (вроде). Необходимо привести к одному масштабу. 
4. Так как фото сделаны сверху - нет строгой ориентации побега (однако, он центрирован) и нет строгого отношения размер изображения/размер побега. Поэтому точно классной идеей будет применить аугментацию изображений поворотом и масштабированием. Возможно, также сдвигом. ImageDataGenerator
5. В качестве доп. фичей можно применить гистограмму цветов в области побега
6. Обучить несколько независимых сетей на частях данных (допустим, 4), делать стекинг на тестовой выборке из их предсказаний. Сам стекинг не обучать (для скорости)
7. Попробовать дообучать несколько слоев ResNet (хотя, эта сеть обучена для слишком большого количества изображений)
8. После обучения смотреть матрицу ошибок, выделять дополнительно то, что можно дообучить в случае больших ошибок сети
9. Подумать над размером и стратификацией батча (видел батчи по 16-20 фото, но у нас 12 классов. Так, есть большая вероятность не попадания нескольких классов в батч)

## imports

In [13]:
import pandas as pd
import numpy as np
import os
import imageio

from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm


# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# global vars

In [14]:
BATCH_SIZE = 16
EPOCHS = 30
RANDOM_STATE = 11
CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

In [15]:
gen = ImageDataGenerator(
            rotation_range=360.,
            width_shift_range=0.3,
            height_shift_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
    )

In [16]:
ImageDataGenerator()

# Создание сети

In [17]:
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

In [18]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act

In [19]:
def get_model():
    inp_img = Input(shape=(51, 51, 3))

    # 51
    conv1 = conv_layer(inp_img, 64, zp_flag=False)
    conv2 = conv_layer(conv1, 64, zp_flag=False)
    mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
    # 23
    conv3 = conv_layer(mp1, 128, zp_flag=False)
    conv4 = conv_layer(conv3, 128, zp_flag=False)
    mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
    # 9
    conv7 = conv_layer(mp2, 256, zp_flag=False)
    conv8 = conv_layer(conv7, 256, zp_flag=False)
    conv9 = conv_layer(conv8, 256, zp_flag=False)
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)
    # 1
    # dense layers
    flt = Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = Model(inputs=inp_img, outputs=out)
    
    # The first 50 epochs are used by Adam opt.
    # Then 30 epochs are used by SGD opt.
    
    #mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    mypotim = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model

In [20]:
# def get_callbacks(filepath, patience=5):
#     lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=1e-5, patience=patience, verbose=1)
#     msave = ModelCheckpoint(filepath, save_best_only=True)
#     return [lr_reduce, msave]

In [21]:
def train_model(img, target):
#     callbacks = get_callbacks(filepath='model_weight_SGD.hdf5', patience=6)
    gmodel = get_model()
    gmodel.load_weights(filepath='model_weight_Adam.hdf5')
    x_train, x_valid, y_train, y_valid = train_test_split(
                                                        img,
                                                        target,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=RANDOM_STATE
                                                        )
    gen = ImageDataGenerator(
            rotation_range=360.,
            width_shift_range=0.3,
            height_shift_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
    )
    gmodel.fit_generator(gen.flow(x_train, y_train,batch_size=BATCH_SIZE),
               steps_per_epoch=10*len(x_train)/BATCH_SIZE,
               epochs=EPOCHS,
               verbose=1,
               shuffle=True,
               validation_data=(x_valid, y_valid),
               )
#     callbacks=callbacks

In [22]:
def test_model(img, label):
    gmodel = get_model()
    gmodel.load_weights(filepath='../input/plant-weight/model_weight_SGD.hdf5')
    prob = gmodel.predict(img, verbose=1)
    pred = prob.argmax(axis=-1)
    sub = pd.DataFrame({"file": label,
                         "species": [INV_CLASS[p] for p in pred]})
    sub.to_csv("sub.csv", index=False, header=True)

In [23]:
def img_reshape(img):
    img = imresize(img, (51, 51, 3))
    return img

In [58]:
def img_label(path):
    return str(str(path.split('\\')[-1]))

In [59]:
def img_class(path):
    return str(path.split('\\')[-2])

In [64]:
def fill_dict(paths, some_dict):
    text = ''
    if 'train' in paths[0]:
        text = 'Start fill train_dict'
    elif 'test' in paths[0]:
        text = 'Start fill test_dict'

    for p in tqdm(paths, ascii=True, ncols=85, desc=text):

        img = imageio.imread(p)
        img = img_reshape(img)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))
        if 'train' in paths[0]:
            some_dict['class'].append(img_class(p))
    return some_dict

In [66]:
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('Data-hackaton'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]

                if ext not in file_ext:
                    file_ext.append(ext)

                if 'train' in root:
                    path = os.path.join(root, f)
                    train_path.append(path)
                elif 'test' in root:
                    path = os.path.join(root, f)
                    test_path.append(path)
    train_dict = {
        'image': [],
        'label': [],
        'class': []
    }
    test_dict = {
        'image': [],
        'label': []
    }

    train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

In [ ]:
train_dict, test_dict = reader()

Root:
Data-hackaton
Dirs:
['test', 'train']
Root:
Data-hackaton\train
Dirs:
['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']



Start fill train_dict:   0%|                                | 0/4750 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "

Start fill train_dict:  73%|###############2     | 3451/4750 [02:57<01:06, 19.42it/s]

In [31]:
os.walk('../Data-hackaton')

<generator object walk at 0x000000000EFDCF68>

In [63]:
train_dict

{'class': ['Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Black-grass',
  'Bl